<a href="https://colab.research.google.com/github/mushhub/my-first-blockchain/blob/main/Simple_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import json
import time
from datetime import datetime

class Block:
    def __init__(self, index, transactions, timestamp, previous_hash):
        """
        ブロックの初期化
        :param index: ブロックの番号
        :param transactions: ブロックに含まれるトランザクションのリスト
        :param timestamp: ブロック生成時のタイムスタンプ
        :param previous_hash: 前のブロックのハッシュ
        """
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        """
        ブロックの内容からハッシュ値を計算する
        """
        block_string = json.dumps({
            "index": self.index,
            "transactions": self.transactions,
            "timestamp": self.timestamp,
            "previous_hash": self.previous_hash,
            "nonce": self.nonce
        }, sort_keys=True).encode()

        return hashlib.sha256(block_string).hexdigest()

    def mine_block(self, difficulty):
        """
        指定された難易度でブロックをマイニングする
        :param difficulty: 先頭に必要な0の数
        """
        target = '0' * difficulty

        while self.hash[:difficulty] != target:
            self.nonce += 1
            self.hash = self.calculate_hash()

        print(f"ブロック {self.index} がマイニングされました: {self.hash}")


class Blockchain:
    def __init__(self, difficulty=2):
        """
        ブロックチェーンの初期化
        :param difficulty: マイニングの難易度
        """
        self.chain = []
        self.difficulty = difficulty
        self.pending_transactions = []

        # ジェネシスブロックの作成
        self.create_genesis_block()

    def create_genesis_block(self):
        """
        ジェネシスブロック（最初のブロック）を作成する
        """
        genesis_block = Block(0, [], time.time(), "0")
        genesis_block.mine_block(self.difficulty)
        self.chain.append(genesis_block)

        print("ジェネシスブロックが作成されました")

    def get_latest_block(self):
        """
        最新のブロックを取得する
        """
        return self.chain[-1]

    def add_transaction(self, sender, recipient, amount):
        """
        トランザクションをペンディングリストに追加する
        :param sender: 送信者
        :param recipient: 受信者
        :param amount: 金額
        """
        self.pending_transactions.append({
            "sender": sender,
            "recipient": recipient,
            "amount": amount,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })

        print(f"新しいトランザクションが追加されました: {sender} が {recipient} に {amount} を送金")

    def mine_pending_transactions(self, miner_reward_address):
        """
        ペンディング中のトランザクションをマイニングし、新しいブロックを追加する
        :param miner_reward_address: マイナーの報酬を受け取るアドレス
        """
        if not self.pending_transactions:
            print("マイニングするトランザクションがありません")
            return

        # 報酬トランザクションを追加
        self.add_transaction("System", miner_reward_address, 1)

        # 新しいブロックを作成
        block = Block(
            len(self.chain),
            self.pending_transactions,
            time.time(),
            self.get_latest_block().hash
        )

        # ブロックをマイニング
        block.mine_block(self.difficulty)

        # チェーンに追加
        self.chain.append(block)

        # ペンディングトランザクションをクリア
        self.pending_transactions = []

        print(f"ブロック {block.index} がチェーンに追加されました")

    def is_chain_valid(self):
        """
        ブロックチェーンの有効性を検証する
        """
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]

            # 現在のブロックのハッシュを再計算して、保存されているハッシュと比較
            if current_block.hash != current_block.calculate_hash():
                print("ブロックのハッシュが無効です")
                return False

            # 前のブロックへの参照を確認
            if current_block.previous_hash != previous_block.hash:
                print("ブロックチェーンのリンクが壊れています")
                return False

        return True

    def get_balance(self, address):
        """
        指定されたアドレスの残高を計算する
        :param address: 残高を取得するアドレス
        """
        balance = 0

        for block in self.chain:
            for transaction in block.transactions:
                if transaction["sender"] == address:
                    balance -= transaction["amount"]

                if transaction["recipient"] == address:
                    balance += transaction["amount"]

        return balance

    def print_chain(self):
        """
        ブロックチェーンの内容を表示する
        """
        print("\n=== ブロックチェーンの状態 ===")
        for block in self.chain:
            print(f"ブロック #{block.index}")
            print(f"タイムスタンプ: {datetime.fromtimestamp(block.timestamp).strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"前のハッシュ: {block.previous_hash}")
            print(f"ハッシュ: {block.hash}")
            print(f"Nonce: {block.nonce}")
            print("トランザクション:")
            for tx in block.transactions:
                print(f"  - {tx['sender']} が {tx['recipient']} に {tx['amount']} を送金")
            print()


# 使用例
def run_demo():
    # ブロックチェーンを作成
    my_blockchain = Blockchain(difficulty=2)

    # トランザクションを追加
    my_blockchain.add_transaction("Alice", "Bob", 10)
    my_blockchain.add_transaction("Bob", "Charlie", 5)

    # ペンディングトランザクションをマイニング
    print("\nブロックをマイニングしています...")
    my_blockchain.mine_pending_transactions("Miner1")

    # さらにトランザクションを追加
    my_blockchain.add_transaction("Charlie", "Alice", 3)
    my_blockchain.add_transaction("Alice", "Miner1", 2)

    # 再度マイニング
    print("\nブロックをマイニングしています...")
    my_blockchain.mine_pending_transactions("Miner2")

    # ブロックチェーンの状態表示
    my_blockchain.print_chain()

    # 残高確認
    print("\n=== アドレスの残高 ===")
    print(f"Alice の残高: {my_blockchain.get_balance('Alice')}")
    print(f"Bob の残高: {my_blockchain.get_balance('Bob')}")
    print(f"Charlie の残高: {my_blockchain.get_balance('Charlie')}")
    print(f"Miner1 の残高: {my_blockchain.get_balance('Miner1')}")
    print(f"Miner2 の残高: {my_blockchain.get_balance('Miner2')}")

    # チェーンの有効性確認
    print(f"\nブロックチェーンは有効か: {my_blockchain.is_chain_valid()}")

# Google Colabで実行するにはこの行のコメントを外してください
# run_demo()